In [25]:
import requests
import pandas as pd
import numpy as np

http_proxy  = "http://proxy-web.micron.com:80"
https_proxy = "http://proxy-web.micron.com:80"
proxyDict = { 
    "http":http_proxy, 
    "https":https_proxy              
}

pd.set_option('display.max_rows', None) #show every row for pandas
pd.set_option('display.max_columns', None) #show every column for pandas

In [26]:
# sii:上市, otc:上櫃
# TWSE之數據是至該季的累計數據而非單季數據, EX: Q3=> TWSE: Q1~Q3, 財報狗:Q3
def financial_statement(year, season, stocktype, type='綜合損益彙總表'):
    if year >= 1000:
        year -= 1911
        
    if type == '綜合損益彙總表':
        url = 'http://mops.twse.com.tw/mops/web/ajax_t163sb04'
    elif type == '資產負債彙總表':
        url = 'http://mops.twse.com.tw/mops/web/ajax_t163sb05'
    elif type == '營益分析彙總表':
        url = 'http://mops.twse.com.tw/mops/web/ajax_t163sb06'
    else:
        print('type does not match')
    
    r = requests.post(url, {
        'encodeURIComponent':1,
        'step':1,
        'firstin':1,
        'off':1,
        'TYPEK':stocktype,
        'year':str(year),
        'season':str(season),
    }, stream=True, verify=False)#, proxies=proxyDict)
    
    r.encoding = 'utf8'
    dfs = pd.read_html(r.text)
    
    
    for i, df in enumerate(dfs):
        df.columns = df.iloc[0]
        dfs[i] = df.iloc[1:]
        
    df = pd.concat(dfs).applymap(lambda x: x if x != '--' else np.nan)
    df = df[df['公司代號'] != '公司代號']
    df = df[~df['公司代號'].isnull()]
    
    #http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.add_suffix.html
    #http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.add_prefix.html
    suffix_format = "_{0}Q{1}".format(year, season)
    
    #return df.add_suffix(suffix_format)
    return df

In [27]:
year = 107
season = 1
howmany_year_data = 3
df_net_income = financial_statement(year, season, 'sii', type='營益分析彙總表')[['公司代號','公司名稱','稅後純益率(%)(稅後純益)/(營業收入)']]
df_net_income.rename(columns = {df_net_income.columns[2]: 'NetIncome(%)_{0}Q{1}'.format(year, season)}, inplace = True)

for i in range(howmany_year_data*4):
    # calculate previous quarter data 107Q1 => we want the previous data is 106Q4 AND then 106Q3, 106Q2, 106Q1, 105Q4
    if (season != 1):
        season = season - 1
    else:
        season = 4
        year = year - 1
    #https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html
    dfanother = financial_statement(year, season, 'sii', type='營益分析彙總表')[['公司代號','稅後純益率(%)(稅後純益)/(營業收入)']]
        
    print(dfanother.columns[1])
    dfanother.rename(columns = {dfanother.columns[1]: 'NetIncome(%)_{0}Q{1}'.format(year, season)}, inplace = True)
    df_net_income = df_net_income.merge(dfanother, on='公司代號', how='inner')
    #df_net_income = df_net_income.merge(financial_statement(year, season, type='營益分析彙總表')[['公司代號','稅後純益率(%)(稅後純益)/(營業收入)']], on='公司代號', how='inner')
    print("year:", year)
    #print("year: {0}".format(year))
    print("season:", season)
    #print("season: {0}".format(season))

df_net_income

C:\ProgramData\Anaconda2\envs\python36\lib\site-packages\ipykernel_launcher.py:34: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



稅後純益率(%)(稅後純益)/(營業收入)
year: 106
season: 4
稅後純益率(%)(稅後純益)/(營業收入)
year: 106
season: 3
稅後純益率(%)(稅後純益)/(營業收入)
year: 106
season: 2
稅後純益率(%)(稅後純益)/(營業收入)
year: 106
season: 1
稅後純益率(%)(稅後純益)/(營業收入)
year: 105
season: 4
稅後純益率(%)(稅後純益)/(營業收入)
year: 105
season: 3
稅後純益率(%)(稅後純益)/(營業收入)
year: 105
season: 2
稅後純益率(%)(稅後純益)/(營業收入)
year: 105
season: 1
稅後純益率(%)(稅後純益)/(營業收入)
year: 104
season: 4
稅後純益率(%)(稅後純益)/(營業收入)
year: 104
season: 3
稅後純益率(%)(稅後純益)/(營業收入)
year: 104
season: 2
稅後純益率(%)(稅後純益)/(營業收入)
year: 104
season: 1


,公司代號,公司名稱,NetIncome(%)_107Q1,NetIncome(%)_106Q4,NetIncome(%)_106Q3,NetIncome(%)_106Q2,NetIncome(%)_106Q1,NetIncome(%)_105Q4,NetIncome(%)_105Q3,NetIncome(%)_105Q2,NetIncome(%)_105Q1,NetIncome(%)_104Q4,NetIncome(%)_104Q3,NetIncome(%)_104Q2,NetIncome(%)_104Q1
0,1101,台泥,11.85,10.50,10.04,9.23,5.49,9.87,9.66,6.89,0.53,7.39,6.79,7.97,4.55
1,1102,亞泥,14.57,10.27,9.55,7.27,1.17,7.68,6.13,5.27,1.49,7.44,8.97,14.19,10.04
2,1103,嘉泥,33.99,46.64,-0.73,-8.06,-18.02,0.31,-4.93,-18.54,-34.92,7.55,-19.65,-4.80,-27.30
3,1104,環球水泥,0.20,31.54,30.60,30.83,25.37,36.42,35.75,36.64,31.02,26.78,24.13,23.91,19.73
4,1108,幸福水泥,-0.74,-3.00,0.80,1.61,2.08,7.41,8.66,9.25,10.48,9.22,10.61,11.71,12.01
5,1109,信大水泥,16.05,13.21,10.15,9.77,4.58,4.09,2.53,2.19,-3.79,-1.32,-3.17,-1.42,0.16
6,1110,東泥,3.72,3.40,0.47,-1.47,6.61,14.04,20.83,29.88,56.46,7.78,8.91,6.35,7.57
7,1201,味全公司,3.27,5.03,1.35,2.02,5.19,-6.35,-3.15,-5.54,-2.34,-10.94,-9.16,-5.92,-6.94
8,1203,味王公司,10.51,12.05,12.47,11.03,10.63,9.93,8.91,8.19,7.94,6.93,9.24,9.57,12.28
9,1210,大成長城,5.40,3.49,3.15,2.73,3.36,3.38,3.59,3.44,3.37,0.62,1.06,0.59,0.45


In [28]:
#檢查column資料型態

#df_net_income.info() #non-null object
#df_net_income.dtypes #object
#df_net_income.applymap(np.isreal) #all False
'''
公司代號              int64
公司名稱             object
NetIncome(%)_107Q1    float64
NetIncome(%)_106Q4    float64
NetIncome(%)_106Q3    float64
NetIncome(%)_106Q2    float64
NetIncome(%)_106Q1    float64
NetIncome(%)_105Q4    float64
NetIncome(%)_105Q3    float64
NetIncome(%)_105Q2    float64
NetIncome(%)_105Q1    float64
NetIncome(%)_104Q4    float64
NetIncome(%)_104Q3    float64
NetIncome(%)_104Q2    float64
NetIncome(%)_104Q1    float64
dtype: object
'''

df_net_income.convert_objects(convert_numeric=True).dtypes #轉成float型態方便後續處理

df_net_income = df_net_income.convert_objects(convert_numeric=True) #轉成float型態方便後續處理

C:\ProgramData\Anaconda2\envs\python36\lib\site-packages\ipykernel_launcher.py:25: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use DataFrame.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
C:\ProgramData\Anaconda2\envs\python36\lib\site-packages\ipykernel_launcher.py:27: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use DataFrame.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.


In [29]:
def isFloat(element):  
    try:
        float(element)
        return True
    except ValueError:
        return False

In [30]:
def growth_func(row, year, season):
    previous_year = year - 1
    #判斷是否為文字而非數字
    if(isFloat(row['NetIncome(%)_{0}Q{1}'.format(year, season)]) == True):
        subsequent_value = float(row['NetIncome(%)_{0}Q{1}'.format(year, season)])
        previous_value = float(row['NetIncome(%)_{0}Q{1}'.format(previous_year, season)])
   
        if(previous_value == 0):
            result = ((subsequent_value - previous_value)/np.abs(subsequent_value))*100
        else:
            result = ((subsequent_value - previous_value)/np.abs(previous_value))*100            
        return float("{0:.2f}".format(result)) # Limiting floats to two decimal points
     
    else:
        return 'NoValue'

In [31]:
def ma2q_growth_func(row, year, season):
    #判斷是否為文字而非數字
    if(isFloat(row['NetIncome(%)_{0}Q{1}'.format(year, season)]) == True):
        previous_value = float(row['Growth_{0}Q{1}'.format(year, season)])
        if (season != 1):
            season = season - 1
        else:
            season = 4
            year = year - 1
        subsequent_value = float(row['Growth_{0}Q{1}'.format(year, season)])
        
        result = ((subsequent_value + previous_value)/2)
        return float("{0:.2f}".format(result)) # Limiting floats to two decimal points
    
    else:
        return 'NoValue'

In [32]:
# use the apply function in pandas to apply the function
# Note the axis=1 specifier, that means that the application is done at a row, rather than a column level
# df_eps.apply (lambda row: growth_func (row),axis=1)

year = 107
season = 1
howmany_year_data = 3

#first 4 quarter don't need to be calculated growth
for i in range(int(howmany_year_data*4-4)):
    df_net_income['Growth_{0}Q{1}'.format(year, season)] = df_net_income.apply (lambda row: growth_func(row, year, season),axis=1)
    if (season != 1):
        season = season - 1
    else:
        season = 4
        year = year - 1

In [33]:
# use the apply function in pandas to apply the function
# Note the axis=1 specifier, that means that the application is done at a row, rather than a column level
# df_eps.apply (lambda row: growth_func (row),axis=1)

year = 107
season = 1
howmany_year_data = 3

#first 4 quarter don't need to be calculated growth
for i in range(int(howmany_year_data*4-4-1)):
    df_net_income['2QMAGrowth_{0}Q{1}'.format(year, season)] = df_net_income.apply (lambda row: ma2q_growth_func(row, year, season),axis=1)
    if (season != 1):
        season = season - 1
    else:
        season = 4
        year = year - 1

In [34]:
# pandas styling
def color_white(val):
    """
    Takes a scalar and returns a string with
    the css property `'color: red'` for negative
    strings, black otherwise.
    """
    color = 'white'
    return 'color: %s' % color

def background_color(val):    
    if float(val) < 0:
        backgroundcolor = '#ff0000'
    elif float(val) < 10:
        backgroundcolor = '#f47721'
    elif float(val) < 20:
        backgroundcolor = '#ffdd00'
    elif float(val) < 30:
        backgroundcolor = '#5ecc62'
    else:
        backgroundcolor = '#00ad45'
    return 'background-color: %s' % backgroundcolor

In [35]:
# styled dataframe can use .to_excel('Styled_Basic_EPS.xlsx', engine='openpyxl') to export excel with style
year = 107
season = 1
howmany_year_data = 3

#first 4 quarter don't need to be calculated growth
columns_name = []
for i in range(int(howmany_year_data*4-4)):
    columns_name.append('Growth_{0}Q{1}'.format(year, season))
    if (season != 1):
        season = season - 1
    else:
        season = 4
        year = year - 1

# the cell’s style depends only on it’s own value. That means we should use the Styler.applymap method which works elementwise.
# Now suppose you wanted to highlight the maximum value in each column. We can’t use .applymap anymore since that operated elementwise. Instead, we’ll turn to .apply which operates columnwise (or rowwise using the axis keyword).
#df_eps.style.applymap(color_negative_red, subset=pd.IndexSlice[:, ['Growth_107Q1','Growth_106Q4','Growth_106Q3','Growth_106Q2','Growth_106Q1','Growth_105Q4','Growth_105Q3','Growth_105Q2']]).applymap(background_color, subset=pd.IndexSlice[:, ['Growth_107Q1','Growth_106Q4','Growth_106Q3','Growth_106Q2','Growth_106Q1','Growth_105Q4','Growth_105Q3','Growth_105Q2']]).to_excel('Styled_Basic_EPS.xlsx', 'EPS_Basic', engine='openpyxl')
df_net_income.style.applymap(color_white, subset=pd.IndexSlice[:, columns_name]).applymap(background_color, subset=pd.IndexSlice[:, columns_name]).to_excel('Styled_Basic_Perspective_NetIncome.xlsx', 'NetIncome_Basic', engine='openpyxl')

# Pandas style function to hignlight specific columns
# http://pandas.pydata.org/pandas-docs/stable/style.html#Finer-Control:-Slicing
# Finer Control: Slicing
# ex: df.style.apply(highlight_max, subset=['B', 'C', 'D'])

In [36]:
#get basic info of company
'''
抓取上市櫃股票的代號、名稱...等資料。
上市
http://isin.twse.com.tw/isin/C_public.jsp?strMode=2
上櫃
http://isin.twse.com.tw/isin/C_public.jsp?strMode=4
'''
#http://bloggerkaiweng.blogspot.com/2016/09/python.html

url='http://isin.twse.com.tw/isin/C_public.jsp?strMode=2'
r = requests.get(url, stream=True, verify=False)#, proxies=proxyDict)
r.encoding = 'big5hkscs' 
dfshow = pd.read_html(r.text)

for i, df in enumerate(dfshow):
    df.columns = df.iloc[0]
    dfshow[i] = df.iloc[1:]
    
df = pd.concat(dfshow).applymap(lambda x: x if x != '--' else np.nan)
df = df[df['有價證券代號及名稱'] != '股票']
df = df[~df['有價證券代號及名稱'].isnull()]
df = df.reset_index(drop=True)

In [37]:
newdf=df[df['產業別'] > '0']
del newdf['國際證券辨識號碼(ISIN Code)'],newdf['CFICode'],newdf['備註']

df2=newdf['有價證券代號及名稱'].str.split(' ', expand=True)
df2 = df2.reset_index(drop=True)
newdf = newdf.reset_index(drop=True)
for i in df2.index:
    if '　' in df2.iat[i,0]:
        df2.iat[i,1]=df2.iat[i,0].split('　')[1]
        df2.iat[i,0]=df2.iat[i,0].split('　')[0]
newdf=df2.join(newdf)
newdf=newdf.rename(columns = {0:'股票代號',1:'股票名稱'})
del newdf['有價證券代號及名稱']

newdf

,股票代號,股票名稱,上市日,市場別,產業別
0,1101,台泥,1962/02/09,上市,水泥工業
1,1102,亞泥,1962/06/08,上市,水泥工業
2,1103,嘉泥,1969/11/14,上市,水泥工業
3,1104,環泥,1971/02/01,上市,水泥工業
4,1108,幸福,1990/06/06,上市,水泥工業
5,1109,信大,1991/12/05,上市,水泥工業
6,1110,東泥,1994/10/22,上市,水泥工業
7,1201,味全,1962/02/09,上市,食品工業
8,1203,味王,1964/08/24,上市,食品工業
9,1210,大成,1978/05/20,上市,食品工業
